# GPU-accelerated image processing with ImageJ and CLIJ
This notebook illustrates how to use [CLIJ](https://clij.github.io/clij-docs/) to push images from ImageJ image variables to the GPU, process them there and pull results back to visualise them. Examples are shown in the [Groovy programming language](http://groovy-lang.org/) but are almost the same for Java or Jython. You can find executable code examples using CLIJ online:
* [ImageJ macro](https://github.com/clij/clij-docs/tree/master/src/main/macro)
* [ImageJ Jython](https://github.com/clij/clij-docs/tree/master/src/main/jython)
* [Java](https://github.com/clij/clij-docs/tree/master/src/main/java/net/haesleinhuepf/clij/examples)

The reference of all CLIJ commands is available for
* [ImageJ macro](https://clij.github.io/clij-docs/reference)
* [Groovy](https://clij.github.io/clij-docs/referenceGroovy)

Please note that notebook might not be the perfect environment for GPU-accelerated image processing. The speedup from using GPUs results from running longer workflows repeatedly in the GPU. This notebook rather serves explaining CLIJs application programming interface.

In order to run the notebook, please install [Anaconda](https://www.anaconda.com/) from its website. Afterwards, please install the [BeakerX](http://beakerx.com/) kernel from the Anaconda command line:
```
conda install -c conda-forge ipywidgets beakerx
```

The following first section is only necessary for running CLIJ from notebooks.

In [1]:
// load ImageJ
%classpath config resolver imagej.public https://maven.imagej.net/content/groups/public
%classpath add mvn net.imagej imagej 2.0.0-rc-71
%classpath add mvn net.imagej imagej-notebook 0.7.1

// load clij
%classpath config resolver clij http://dl.bintray.com/haesleinhuepf/clij
%classpath add mvn net.haesleinhuepf clij_ 1.1.4

Added new repo: imagej.public


Added new repo: clij


## Getting started
The following code instantiates ImageJ. This is not necessary if you run it from ImageJs script editor. 

In [13]:
// start ImageJ
ij = new net.imagej.ImageJ();

net.imagej.ImageJ@1287a47c

Independent if you run this example code from the script editor or from a notebook, you need a `clij` variable giving you access to the GPU.

In [14]:
// start CLIJ
clij = net.haesleinhuepf.clij.CLIJ.getInstance();

net.haesleinhuepf.clij.CLIJ@2199df72

## Loading images and sending them to the GPU

In [15]:
// load an image
image = ij.io().open("https://imagej.net/images/blobs.gif");

[INFO] Verifying GIF format
[INFO] Reading dimensions
[INFO] Reading data blocks


In [16]:
// push image to the GPU
input_image = clij.push(image);

// reserve memory in the GPU for the result image; same size and type as the input image
blurred_image = clij.create(input_image);

ClearCLBuffer [mClearCLContext=ClearCLContext [device=ClearCLDevice [mClearCLPlatform=ClearCLPlatform [name=Intel(R) OpenCL], name=Intel(R) UHD Graphics 620]], mNativeType=UnsignedByte, mNumberOfChannels=1, mDimensions=[256, 254, 3], getMemAllocMode()=Best, getHostAccessType()=ReadWrite, getKernelAccessType()=ReadWrite, getBackend()=net.haesleinhuepf.clij.clearcl.backend.jocl.ClearCLBackendJOCL@6bda62d6, getPeerPointer()=net.haesleinhuepf.clij.clearcl.ClearCLPeerPointer@19e14b6c]

## Filtering images in the GPU

In [17]:
// apply a Gaussian blur
sigmaXY = 5; // in pixel units
sigmaZ = 0; // in pixel units
clij.op().blur(input_image, blurred_image, sigmaXY, sigmaXY, sigmaZ);

true

In order to show an intermediate result from the script editor, you can call 

```
clij.show(blurredImage, "blurred image");
```

However, when working in Jupyter notebooks, you need to wrap or convert the result like this:

In [18]:
// show intermediate result
intermediate_result = clij.pull(blurred_image);

// the image needs to be wrapped/converted in order to be visible in notebooks.
import net.imglib2.img.display.imagej.ImageJFunctions;
ImageJFunctions.wrapReal(intermediate_result);

## Apply a threshold algorithm in the GPU

In [19]:
// reserve memory for another image in the GPU
binary_image = clij.create(input_image);

// apply the threshold algorithm
threshold_algorithm_name = "Otsu";

clij.op().automaticThreshold(blurred_image, binary_image, threshold_algorithm_name);

true

In [20]:
// pull image back from GPU memory
binary_result = clij.pull(binary_image);

// wrap and show the image
import net.imglib2.img.display.imagej.ImageJFunctions;
ImageJFunctions.wrapReal(binary_result);

<img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAQAAAAD+CAYAAADVndu7AAAL0ElEQVR42u3dW5bdOA5EUXn+g+6eQHfZrpRIRGB/8C9XXokCDkk8gs+vX7/+89PxPM+PxxvP8fYzffVsXz3njXdIe9628eO5BgAAAAAAuPoQ0w0TAAAAAIJW/7eNEwDuPD8HPzD3jat/ApwaAJAKLQB4CQDtDhbzEQEAABIBMH2SAAAAqp3/JwBI+riTnw8A5gAgCUQA8NHzxn5IAPj8XQAg1PknUx4AsnZgABAOgNqtHAAce/4qm2mM/AMAADQEiCsBwOnPOz8A7DjGAgAAAAAAfAcA238AaOgDSLfjWABwejGApvJrAAAAAAAAAAAAANgS/V97BOD8AAAAhbUASQBoBEuyFkCS8ErL9n8lANp3GOk19c2lv2886xfvGwWAUy8KAHldl4kNWBMWrqgg4LZGDdp6v159rrffd4rzn1wcrwGgtY8/AQKb4yO3bWviAnm8FHhjp5YgVU5g8eSZf+wuAAD6HAAE3zkWpBxdx+kBbO7V3t6rvj0Wcuv5xoiCcgByVQAwp07md7/3WAF/9iyqGaVDJ9vtb/+myRhuPgMHBoDEnevTZBCOHwAAAEsBIAIPAABQBoDTATDFNwDQcnytAcApPX8VeACw8a6Bh+EpwQUAAGB0AAAAF97l+pwzOgBQBHTnnUbMOYMDAN9j8ZwzOAYnHgMADI3RAcCirBcAAABNhOB3B4DFIpy+z27nf7WDkIEBwN/OF1GUngpCAACAsRoEzvwAAACLujDTi2qmCqACAABUdLPRgQwFQMsFFpx/X+wDAArPcAAg8j5tl1EHgA33w3P+XTsgACgp4gAA1XfTjqkAcHDCOT8AtABgVBrQvfacP3E+km1yDAASt9WCXo5DyfGpUaXAqYbkCjIASMxQjWsG2iLnrCiqZ55SdQXHtQM35tW3dPYBQIYtjhYEUVgDAGnfjS0CgLEUAGwRAAwAYIcAYHB+AAAAEFjxzdghAJDeAgB2mAIAK3Bei/R250+bGwCQhwaAF94pdX4AQCS6vjfi6/dKBPn4UmDOrzciAQAT5utH7wcAAJBaInsbABO+4499zTmS9l6KmOs04N7+hq/4m3NkXr59wnMn/ea01f8VKa+3dtvOkQDQnhKdtvrfBNoRADSeI0mQZR87bkf+b8PsOADazpEAkC3KAgCXAHD7LAkAAPDFYlSx/T8NgOnGpta+GwBvluoCAAMDgMW6jBXb/00AiK3U+kILPvzClq0FXQAQmpacAICEXUja7ic6/z8BAFOMKqJia9rKceF7aeoqAkDSypsKgC+FWk5DwNGyCABpK2/qs5wAANm2X5FHytfbgb94eIG3+c4PAHeOL299r6d1y7Q58n7S+QEgO/7wJGxdWqLeAGBMO348CatgU9rrxG9zfkD4i7+f75Btee+4clEAaAbGE7EqtRW+cH4jGgDTmye2GjvnlyU4BoBT7ZMacGZB4PS7/Om7qhUAABJc4fc0NIJ7OGDPTDIAZAPArqVTH/M5ObkbKvCoDwlcht0BcW5i2wtw3D8w2/nTbl2uA8DJXoAp8NmsgzDR+QHgMgBuCj9sS3MlRPibIRASe5kjVtHUirxBaz/V+ZOCmvUAmOAcasV/rQPAlGu6Bzyf6jBjn/NvBcD/+B0AMACgHQD/8DsAYABAMwB+8zsAYABAaxHQH/wWABgA0AiAP/wtzm/IAjTWYgBAiWMAwJ6KwAu/CQDaRXdCYNK7XpxbAFCvvw8A7OpFAHBkrcoaggDAKO7Z1xJcO58mMt3gzQmbvQIAzrxHsouoqSPA9X59xr3PqM3TQAAQ6pi5ujFozv8pAKTBegGQMsec/0IlIKL3AiB5jjn/C3OU7AQAcOb6tMR5FWz+w+8sB74TALbQ8779ld+WAtuXBpyoUOMsfgkEUmC7ADC9R1324nC78rbtb6MsdoJBqmwcKl22pbqrVRv/5O9pee4rYFoBAJdH3k29qvWYC4EaAPzTBLVdI51UeyG+MxsCT4vz/7/JUTV2t/YCAGZDAAAAoPLY0dzSDAAAsGL7n16NmAABAACACAC09CUAwOAAIADMBsDkb5Mqa/ZsCIxsB0Dy+b+9TXkFAG5XAVr97xnbFggAwPAyYLJaXQCY9r0AIORMyPlnlwCnQgAAwqLC2lDPGdoGCNQBYMClhUQqSmTXACAgDZhmEBw8a47bIRALgMRVgYPnAXbDjT9rAHATAhw8V3AVAJ6JO7MRFOL4S+TWmyGQuMt5pm8TOXVXbMUuYNyCKxjH+Q8LUcoITDpyA8DN8++m1X9TdeCkd/rsevB0AEwKdG5LrSoPHhNo3wmAJiXcROfZdkfExMD6jwGw5cw8XQk30XFcEjNmbq3+6ZdvpDoOAIyYU7fzpl/Akeo0nH/EfFr9T+sOTNsFiJavTgMDwGYAKGBaXwMi+HdDegwA9l4eMgyeLm1sAEC6+CnnDwDA9glLyMfLl3P+vxo3ymIbARBS9x37rTj8R/Zm4mat/ozdOJo+NXkAsPnbr3b+jQD4KlgGALubvgBgeTPQpo42TV9FYiNWhhnlv6mqNgAAAFUw2FzSuvnG3pXODwCaWpqr8QDgEAC2B6/SAUCNBwBWR1Y3A4Ac1/e3IFUBQEoLAADg/KI4AgDy2l0AIMh5XpA1FgBTG1w4P0muJADclH371wBI6HEHAM6fIsc+6bj2TI0Sb1W29ez9dzFMUk56JgaJbKE9e0LlX4Loy2/b2KdtcbdfbwUAOYU/DaIvzyQjccGlDEBLyu/W8eOv/w8ADIrIAkBM70JNvGTSCpdikACQ3bvQ8Pt1AEhxpC+dPxEAm/sWAAAAVhcCJaddAQAA1gMgyYgnznNDWfzTFgD8cqLbnH9zxeUNCIzsY5k04QBAODMx5Xo67RcJgIbIdDMAVophfKwD8HXBDwAUAYCjg+6VTsaWHgAAAIBtIiav7DKmtQEDgHESANuvSP+0GWhKy6P6f86/QVTmEwDcaLbYVAbM0XcD4PZ7PNPSHdPrrAEAAJps6Wn6+OroAWAzAF5vB3azS16cAgAAoAYAk1WBOX8PAFr0JGsB4JhifgEAABxTlj1z8xEAAJbAoGVFBQAAMGylAQAAjO3naADo+S4AwPlXQ6Dp+wCAUec8AHC5Gcjg/K0AEJsBAM6/FAJN3+ozPQADAFxk0p2eBQAAWJUWbPlmr/0eh+H8pMxzvt/rv8FpAEBj02IbMSEAAACL7cOkAAAA/Hx+AcB4/eMAwL7z+fHnn2zMVuu7dysAwHk9zHgAMJi7uV8AAIBrAGA09wtaAEDK8QoAGM79HnDODwAxAJhURdYaoQcAO9sjWYCWRhLOz/kBoEhrbsozcX4AqK4EbLzIEwAAoDm7VQ2AaYbO8VUBjn2fttUWADj/KQhUvI/t9uyrzr94do5vAMApySWrtgEA2Vvt1F0AAzf+wDZnGDgAcH7jEgBekRcuBkDiMYBx92YePgXArT50AAAA43yQ9nnrnwOAPn0jr57mmSREAQAAkNRnMi17dRUAE3YB0x0KAHLTuckaET8CwMldAAAAwKl5Arq/AMApCLQD4CQEOH/X5SZfvVMVACbXApw0cs6/45qzkQC4HbgBgIfzL7vs9Cfv8nxEFQAoec6Ncmkt73oNADc1ApMMIhVUGyTTACAUABNbgqemgKbo4G/TQJj0Ds/HOcb4M3ZzEciU77ZJCGXa8z9fTwZZsCxp9W2iKQBwpt44cpu1TYV2o2oSAJRGmBsrxRqbUQAgBAAaSXSiAQAAuNgBAFYdA2QB9JYDAAAAgBtaAAAAFhYC2coDwNZ6gPW9AFt10gAAANJ7An7cDZhunJw6Jw24tStwLAAaDJNDZwJgCgSSOyH/tSKQXD4AbIyUT7Cda6KgjNKYutvaeJ/CcVlwADB+Mt9NQiEbM16PFcmYnm0hofbh3AKAMaWFu0noBQAAoB

## Multiplying pixel values in the GPU
The binary image contains values 0 and 1, which are hard to differentiate without being able to change brightness/contrast. Let's multiply the image with 255.

In [21]:
// reserve memory for another image in the GPU
multiplied_binary_image = clij.create(binary_image);

// multiply all pixels with 255
clij.op().multiplyImageAndScalar(binary_image, multiplied_binary_image, 255);

// pull result back from GPU
multiplied_binary_result = clij.pull(multiplied_binary_image);

// wrap and show the image
import net.imglib2.img.display.imagej.ImageJFunctions;
ImageJFunctions.wrapReal(multiplied_binary_result);

<img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAQAAAAD+CAYAAADVndu7AAAME0lEQVR42u2dR3IsOQwF6/6X/rObzRi1ukjgmUSEdoouEiZJgO75c0Ce53n9d1pOtOlW2261c6MPbu1Nk9e6BgAAAAAAgNVGqDsmAAAAAMBo9D/tnABgp/3IgO4TR38HOCUAwBVaAOAQANIDzMaIAAAAOAJAXUkAAABEB/8bADgZV7l9AEAHAE4gAgCX2mtrSABwvS8AwDT4lSkPALxmYADAHACxUzkAMNb+KJ9JrPwDAACQUCCOBAAyH/wAoCONBQAAAAAAgHsAYPoPACba76B3AEDwUwMoBkDMyhEAAAAAAAAAAAAg0XYXndelAAQ/AAAAgXsBnACQCBbnuwBOt99J3wBgUGHpMwz3PfXJW39PtPVGf60AMNVRAOB36tLxAJbCwGVVBJzqJLMAn0r15iCgEvyTg+MaAKY7CASoj3zS99v6VBwgx7cCN0wnKVL5FhYnc37ZWQAAyAsAIHgmLXBJXeXuA2gqKLUX1KiFaNhN5lJQAmDHqdrB1w6Ab2fnT5ojTLcF0Q/+FgD8W1t+/J8kZ9hsAwIAHGeuT5JDkH4AAABQCgAVJ0QAAAAwLYCx+QYApKSvMQD4SamKzocAAJu7BXA7tuACAACA0wEAALDQl3Wd43QAIB0Aqn2S0DkOBwCwR7HOcTgcjnoMAMDRcDoAYKAHAAAAsE2wHcY3vuFgACDRPg39BgAAYEVfDjZq8UsAAADWdJQYEG4+CQAAQMRUVCkvdvNJAAAA7EeiN21r90l5AKQ8YEHw99U+AEBgDgcA7ugkeSQGACYd2gQAKVH2DAgADDoTAPBNiVJ1AABCRhOCHwCkAOD1d1sdieAHht/oIK1m9ig7kjoEkoteLbpxrk8d+V67I/EEGQBwXKE69j0caUfxAIDR/9u2Hf2OshOtXpdccrIPAHj44rXfBgDd0mo3fBEAIKUAwBcBAAIA8EMAgBD8AAAAAIEKm+GHAMAqEAEAAKgGACOw3xHp9uB30w0AMHJKAKALAFf9AAAjh1SfBaTWOBJBfhQATXlkei5K5f93fVPQ16v+AQAAoDyiKQNAwY6vY4080ssZ1VYHmoG7bcMj8UYe6RX8GzpSSUWUALBtx2OzbfJIAOCkIxV/3LTj0XSbPNIr+JklPRL+uGXH46tt5JEAwE1HAMAIANu5JAAAADcGo4jp/zQA1J3NIfgBwL3imUOxGwCIOxgA8LPPhk0lgr8JALY7tW7cBW/+YEtCWgcADAyVBACHWYjb7Md6/V8BACpOZbFjS23kWLBXyuACAIYMnTLl3ipUKUGA1DIIAG4jr2tbJgDAtW1/LFPKv39LmZgU3vSDHwDspC+n7CUNAKX1UqdaxGTwAwDv+sPjMHVJqXoDAEQt/XgcRsEUAExVjQl+gPCL/9cPyCQATBSNAAByFQAbo1ISAD5pH8GPSANA/fBEq7MT/KwSjAFg6vhkWgrgDoHpvnza1+RiHQBYDITUynDCjcSJJwetAHD65RQAoAkAZi2z/YoDwLfKUHCsZgAk1S1UblQW0/2cYtM34KRBIC343V5djgPAb5WybYzkqnNCgAAAQwB8qphtg6QvO6lU+JMhYFJ70bmsQsUoaZKWGzcVNeMBoBAcTaJiq20AqDzTLdA+dochfcHfCoB/+Q4AQABAOgD+5zsAAAEAyQD44TsAAAEAav0cbAMAQABAIgA+/BbBj3RCQLWfw98GAOqBAQB6dgQufBMAuAUDEPC810D02nAA4BgEAAB/lQEAshcA6ACfBQDljo8u8NkX+kSR7g6PTvDZFQAgOs6OfvDb0RQgoVKa5Nw4MsE/BoBNI+PQ5LXoZwkASoYFAPefU0/SH6nmCwBA9FwAOOuY4D+gI+cgAAAzz6c56hX50M7uQQAAZt5qRO7bfuXbCQEABGYeamXEnrH3aFsSRj8AoP1GY0rNYdrWI+1rm/4mXovt4JDsbBS9uiwh+D9RUurd+JPfS5kGO/o7AHihIB6P3AGOUs2BVDccAP+noLRnpF1mHNuzjqTgv7YLNyX4/0s57Bo7awMAoDvQAQAAEAcAJ3s4HmcGAABAOviddyM6QAAAAAALAKScSwAAwgVAAKANAGXbuF5rFg+Ab75P8OsCIO2YcgUAbivo5LcZ/fWWHh3sBACEAfDp91MlEQBq9gIAJjkhwa+9BdgVAgDArCrMMdQ5R2uAQBwABB4t5JIK89EfABgtA7o5BOKl43QI2ALAcVRA/ADb8OJPDQA2IYB45P3MAkzOAohQiMAPHv1bIOA4y3nUp4lIVm2FWYDcgEsxjuAfvoiSFQGllBsAbOa/TaN/CwBO9/FyW3oBoFTobALATQgkzcqG2tAJgKSbcB2Dp+2NCMXC+msAtOTMkw7eMPo3AkBYt4z+7o9vuAYOAJDQKa/zuj/A4Ro0BL+EPhn9p+8dUJsFUC2vDX4A0A6AhGo5wf/MAqBVWcrV+GYAEPyDAGhWmDIAvvl+W20GALwEQLvC1AHwmza42wc55G8//ShKu3P1+NYOMFdbIZf8DcVpjf44OzLpawAAAFTbvjr4GwFwq1gGAHztDQBwitfO0XSirdnOUcHfDIDTI0PrrTYAAABEwUDBOGn1EUAmGvwAQNM4jn0AAMUAaC9euQOA23gAQHVltRkAXMd1/xWkKACwpAUAAMD8oCgBANa1swDAhZzzF7LaAkD1gAvBrzWqAADda9++BoDDGXcAQPB/078NPSqla8/kh1UV2xhEjP57bzFs2uwfv6lYJGIKTdvVg3/LT48fY1eb4rY/bwUAPPL+zYHq6CE2JSfhgUtWAFKW/LbSj1//DgAQqsgCAOkc3HGm+jUANhzcxSEBgPfZhYTvxwHAJZBuBr8jAFxXXtxnIAAAANRNY10uXgUAACA+qJyDfxMAajIKAPeiVFrwby2fJS1b2t+ZoKRwAKC7lOa0zDet7+llP0sAJFSmkwGgLC46n97wAwCCAIAA3Y204lEKRgCANANgZVvxbeVvGRwAAACeSH8JgLcN2zJ4azGK4M8/xHQ83m400JWGG06JdANgux/H7wTcVoqbUyK9AFDox5Nk/IYpGwAAAEdrbikO4GY4BAAAgAOKcjQeogWAlPskYwFAmoJ+AQAAIE0pa3NyCgAASmCQMqICAACAMJUGAAAAac+jAUCOXQAAwV8NgST7AAAkLngAwPJhIITgTwUAtRkAQPCXQiDJVq++T7gAAB4y8bXX628TLgCgaVkwxWbHvke4EPxcZe5jv+PfIGQAAAebin0EdQAAAFDsH6gEAACA9/oFAMhx4wCAvvx8vP3Kzsxovfu2AgD4nV4BQAAlHabqAAAASAIAp5kdoQGA70wtDgA4zt70nOAHAHYAUNpF5g6ArT8A4KePKwBIOUhC8BP8AECooJUCJoIfAMj2JxUASk4OAPIhYNufRACoOTqBnw0B6/6kjbYAgOCfgkBEf5huaz91fqPtBD4CAKauXGLURgCA91TbdRaAIB/4poaDAwCCH1kCwJHrhYMB4JgGIHlFx0sp86EXRgAAAEDG/fgKAL75cQCgBQEkO/APztx1LqIAAADglN6agv/lzP3cjwIAHQg0j4bpbwIcfR5c6ToqAAAApvQE6H4BgCkIpANgEgIEf9bjJrf6FAWAm0p1cnKCv+OZM0kAbBduAMBD8Jc9dvqmL88lqgCAkHa2AWBCh0r9eC4WF2IA0FzpdrvDwQECAMCs0Ja6BPS2XanB7wj+b/vwXF5jtM+xXaa/W+1JBkDC/ZCrAJgeSW4oWj0PTmgDAAgGwE+dbhlVb7RdRWcAAADEVZUV+6GsMwAQDoCmpaVGab02DQCwt5wjtH+yz3GkXls/BgAEAAAAAGDhqAgAAAAHNwIxlQcArfsBlPU9AgBXB0UAQPuZgF+0ISv4gcCOQ7pWyd13gF4BQIJjIp4AUIGA0+zr2I

## Cleaning up memory in the GPU
When working with GPUs, it is important to release memory afterwards:

In [22]:
input_image.close();
blurred_image.close();
binary_image.close();
multiplied_binary_image.close();

null

Voila! When running this notebook, please try to stick to the top-bottom order, otherwise you might overwhelm your GPU. Just as an example, if you call a block with `clij.push(...);` again and again, there is more and more memory allocated in the GPU. Notebooks might not be the ideal environment for running GPU-accelerated code. This notebook rather exists for illustration / education.

Feedback is welcome: rhaase@mpi-cbg.de

Happy coding!

Cheers,
Robert